<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [4]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [8]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [10]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 人気3位以内にマークを付ける
ldat["high_prev_popularity"] = 0
ldat.loc[ldat["prev_popularity"] <= 3, "high_prev_popularity"] = 1

In [16]:
ldat

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float,high_rank,high_prev_popularity
0,1.0,6,6,0,0,2,54.0,0,1:31.6,0,3.2,1.0,474.0,0.0,0,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,3.0,6.0,6.0,0.0,0.0,2.0,51.0,75.0,1:12.2,1.0,9.6,3.0,474.0,-2.0,0.0,1.0,9.0,0.0,0.0,-,1200.0,1.0,0.0,10:40,2006.0,7.0,29.0,True,3.2,1561165932,1561200000,1561200000,1,1
1,2.0,7,8,1,1,2,54.0,1,1:31.6,1,6.4,4.0,448.0,-8.0,1,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,3.0,1.0,1.0,1.0,1.0,2.0,54.0,1.0,1:57.7,18.0,26.9,7.0,456.0,0.0,1.0,4.0,9.0,0.0,0.0,-,1800.0,1.0,1.0,12:30,2006.0,7.0,23.0,False,0.0,1561168662,1561200000,1561206600,1,0
2,3.0,7,7,2,1,2,54.0,2,1:31.9,2,4.3,3.0,476.0,0.0,2,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,4.0,1.0,1.0,2.0,1.0,2.0,54.0,2.0,1:12.5,1.0,6.7,3.0,476.0,-8.0,2.0,1.0,9.0,0.0,0.0,-,1200.0,1.0,0.0,10:40,2006.0,7.0,30.0,False,0.0,1561165950,1561200000,1561200000,1,1
3,4.0,2,2,3,1,2,51.0,3,1:32.1,3,33.1,7.0,484.0,0.0,3,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,12.0,1.0,1.0,3.0,1.0,2.0,51.0,3.0,1:57.3,7.0,11.9,5.0,484.0,0.0,3.0,5.0,9.0,0.0,0.0,-,1800.0,2.0,2.0,12:20,2006.0,7.0,2.0,False,0.0,1561168638,1561200000,1561206000,0,0
4,5.0,5,5,4,1,2,54.0,4,1:32.1,1,3.9,2.0,518.0,6.0,4,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,3.0,2.0,2.0,4.0,1.0,2.0,54.0,15.0,1:12.5,9.0,6.6,2.0,512.0,-2.0,4.0,1.0,9.0,0.0,0.0,-,1200.0,1.0,0.0,10:40,2006.0,7.0,30.0,False,0.0,1561165950,1561200000,1561200000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542850,9.0,8,13,42682,1,5,57.0,415,2:02.1,10,58.6,9.0,468.0,-2.0,94,12,9,0,0,-,2000.0,1,0,16:01,2017,3,5,8.0,1.0,1.0,42682.0,1.0,5.0,57.0,387.0,2:01.9,4.0,23.3,8.0,470.0,-2.0,94.0,9.0,9.0,0.0,0.0,-,2000.0,0.0,0.0,14:15,2017.0,2.0,25.0,False,0.0,1561168914,1561219260,1561212900,0,0
542851,10.0,5,6,44140,1,5,57.0,418,2:02.2,1,140.9,12.0,494.0,-2.0,116,12,9,0,0,-,2000.0,1,0,16:01,2017,3,5,8.0,1.0,1.0,44140.0,1.0,5.0,57.0,418.0,2:01.3,1.0,113.1,10.0,496.0,-4.0,116.0,10.0,9.0,0.0,0.0,-,2000.0,0.0,0.0,14:40,2017.0,2.0,19.0,False,0.0,1561168878,1561219260,1561214400,0,0
542852,11.0,5,7,45594,1,5,57.0,497,2:02.3,4,14.3,6.0,496.0,2.0,192,12,9,0,0,-,2000.0,1,0,16:01,2017,3,5,2.0,5.0,5.0,45594.0,1.0,5.0,54.0,497.0,2:01.3,9.0,19.5,6.0,494.0,-4.0,192.0,9.0,9.0,0.0,0.0,-,2000.0,0.0,0.0,14:15,2017.0,2.0,25.0,False,0.0,1561168878,1561219260,1561212900,0,0
542853,12.0,4,5,52157,0,4,55.0,254,2:02.6,2,174.9,13.0,460.0,-4.0,450,12,9,0,0,-,2000.0,1,0,16:01,2017,3,5,8.0,3.0,3.0,52157.0,0.0,4.0,54.0,246.0,2:03.0,3.0,61.0,10.0,464.0,1.0,450.0,10.0,9.0,0.0,0.0,-,2000.0,0.0,0.0,14:50,2017.0,2.0,26.0,False,0.0,1561168980,1561219260,1561215000,0,0


In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("high_rank == 1")

In [0]:
loser = sdat.query("high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1).reset_index(drop=True)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
x = sampled[['high_prev_popularity']].dropna(0)

In [0]:
#y = sampled[["high_rank"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["high_rank"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"])

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
testorg = sampled.tail(len(x)-trainlen) 

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 5


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [45]:
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               1024      
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [47]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 112774 samples, validate on 112774 samples
Epoch 1/5
112774/112774 [==============================] - 9s 82us/step - loss: 0.6631 - accuracy: 0.6068 - val_loss: 0.6616 - val_accuracy: 0.6106
Epoch 2/5
112774/112774 [==============================] - 9s 80us/step - loss: 0.6628 - accuracy: 0.6074 - val_loss: 0.6619 - val_accuracy: 0.6106
Epoch 3/5
112774/112774 [==============================] - 9s 80us/step - loss: 0.6627 - accuracy: 0.6074 - val_loss: 0.6612 - val_accuracy: 0.6106
Epoch 4/5
112774/112774 [==============================] - 9s 78us/step - loss: 0.6627 - accuracy: 0.6074 - val_loss: 0.6619 - val_accuracy: 0.6106
Epoch 5/5
112774/112774 [==============================] - 9s 78us/step - loss: 0.6626 - accuracy: 0.6074 - val_loss: 0.6622 - val_accuracy: 0.6106


In [0]:
predictions = model.predict(testx)

In [49]:
print(predictions)

[[0.5863744  0.41358227]
 [0.33212164 0.668324  ]
 [0.33212164 0.668324  ]
 ...
 [0.33212164 0.668324  ]
 [0.33212164 0.668324  ]
 [0.33212164 0.668324  ]]


In [50]:
score = model.evaluate(testx, testy, verbose=1)


112774/112774 [==============================] - 4s 34us/step


In [51]:

print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.662206160921131
Test accuracy 0.6106194853782654


In [0]:
predict = pd.DataFrame(predictions, columns={"lose","win"})

In [0]:
result =  pd.concat([ predict.reset_index(),testorg.reset_index()],axis=1)

In [79]:
result.query("win > 0.5").groupby("high_rank").sum()

,index,win,lose,index,rank,frame_num,horse_num,name,sex,age,amount,jockey,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,distance,weather,condition,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_distance,prev_weather,prev_condition,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float,high_prev_popularity
high_rank,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2704453196,28124.861328,19837.060547,8113545332,458965.0,228160,384970,1227912199,30206,177040,2636065.5,7565910,338443,4393389.8,457616.0,22389004.0,9626.0,8591919,302756,422000,26609,32664,74497130.0,24402,26468,96473491,310236,755284,369493.0,198407.0,333755.0,1.057945e+09,26672.0,155033.0,2306313.5,6489358.0,282913.0,3453175.0,399186.0,19588004.0,13234.0,7429956.0,273787.0,366861.0,22377.0,28434.0,65374620.0,21052.0,22020.0,84380606.0,272688.0,662573.0,0.0,0.0,6710770718649,74881796106540,10055702037713,0
1,1993361243,20733.613281,14623.855469,5980937109,73172.0,167590,270705,909825016,24181,125413,1951208.5,4936775,167932,743526.9,183323.0,16718468.0,10690.0,6208075,223922,312207,19520,23044,55169040.0,18021,19232,71118051,232358,556163,187466.0,143293.0,240135.0,7.591083e+08,20902.0,108893.0,1659140.0,4291518.0,191198.0,1162433.8,225476.0,14170376.0,18183.0,5240689.0,202366.0,259550.0,15980.0,20257.0,47683710.0,15152.0,15911.0,60427448.0,193199.0,474155.0,10545.0,152319.4,-4394034875874,55202766638640,-2117252382715,0
